<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #61afef;">
<!-- BAŞLIĞI BÜYÜTMEK İÇİN font-size DEĞERİNİ ARTIRDIK -->
<h1 style="color: #61afef; font-size: 32px; text-align: center; border-bottom: 2px solid #61afef; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
🧠 Bölüm 1: Proje Ortamının Kurulumu
</h1>
<p style="font-size: 16px;">Bu bölümde, <strong>Nörobilim Asistanı</strong> projemizin çalışması için gerekli olan tüm temel altyapıyı hazırlayacağız. Süreç, projemizin sağlam bir temel üzerine inşa edilmesini sağlayan iki kritik adımdan oluşur.</p>
<h3 style="color: #e5c07b;">1.1. Gerekli Kütüphanelerin Yüklenmesi <font color="#98c379">📚</font></h3>
<p>Projemizin bel kemiğini oluşturan Python kütüphanelerini Colab ortamına kuruyoruz. Bu adımda, tüm bileşenlerin birbiriyle uyumlu çalışmasını garantileyen belirli versiyonlar ve kurulum yöntemleri kullanıyoruz.</p>
<ul>
<li><font color="#61afef"><strong>LangChain & Ekosistemi:</strong></font> RAG mimarisini ve chatbot'un ana mantığını (zincirler, hafıza vb.) oluşturmak için.</li>
<li><font color="#c678dd"><strong>Google Generative AI:</strong></font> Projemizin "düşünen beyni" olan güçlü <code>Gemini</code> dil modelini kullanabilmek için.</li>
<li><font color="#e5c07b"><strong>Vektör Veritabanı:</strong></font> <code>ChromaDB</code> ve ilgili <code>LangChain</code> entegrasyonları, bilgi bankamızı verimli bir şekilde saklamak ve içinde anlamsal arama yapmak için.</li>
<li><font color="#e06c75"><strong>Hugging Face:</strong></font> Metinleri matematiksel olarak anlaşılabilir vektörlere dönüştüren <code>embedding</code> modelini çekmek için.</li>
</ul>
<br>
<h3 style="color: #e5c07b;">1.2. Kurulum Doğrulaması <font color="#98c379">✅</font></h3>
<p>Kurulum tamamlandıktan sonra, yüklenen tüm kütüphanelerin Python tarafından doğru bir şekilde tanınıp tanınmadığını bir <code>import</code> testi ile kontrol ediyoruz.</p>
<blockquote style="border-left: 4px solid #56b6c2; padding-left: 15px; margin: 10px 0; color: #abb2bf;">
🚀 Bu testin başarıyla geçmesi, geliştirme ortamımızın <strong>sağlıklı</strong> ve projenin geri kalanına başlamak için <strong>hazır</strong> olduğunu doğrular.
</blockquote>
</div>

In [9]:
print("--- ADIM 1: Gerekli modern kütüphaneler yükleniyor... ---")
!pip install -U -q \
    langchain \
    langchain-core \
    langchain-community \
    langchain-chroma \
    langchain-huggingface \
    langchain-text-splitters \
    langchain-google-genai \
    chromadb \
    datasets \
    google-generativeai==0.8.5 \
    pandas==2.2.2 \
    requests==2.32.4 \
    gradio

print("✅ Kurulum tamamlandı ve Colab ile uyumlu hale getirildi.")

print("\n--- ADIM 2: Güncel import testi ---")
try:
    import os, time, gradio as gr, pandas as pd, google.generativeai as genai
    from datasets import load_dataset
    from langchain_chroma import Chroma
    from langchain_huggingface import HuggingFaceEmbeddings
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain.memory import ConversationBufferMemory
    from langchain.prompts import ChatPromptTemplate
    from langchain.chains.combine_documents import create_stuff_documents_chain
    from langchain.chains import create_retrieval_chain
    print("✅ Tüm import'lar başarıyla yüklendi! Ortam temiz ve uyumlu 🚀")
except Exception as e:
    print(f"❌ Import hatası: {e}")


--- ADIM 1: Gerekli modern kütüphaneler yükleniyor... ---
✅ Kurulum tamamlandı ve Colab ile uyumlu hale getirildi.

--- ADIM 2: Güncel import testi ---
✅ Tüm import'lar başarıyla yüklendi! Ortam temiz ve uyumlu 🚀


<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #c678dd;">
<h1 style="color: #c678dd; font-size: 32px; text-align: center; border-bottom: 2px solid #c678dd; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
🔑 Bölüm 1.2: API Yapılandırması ve Model Testi
</h1>
<p style="font-size: 16px;">Kütüphaneleri kurduktan sonraki bu kritik adımda, projemize Google'ın güçlü yapay zeka modellerini kullanma yetkisi veriyoruz.</p>
<h3 style="color: #e5c07b;">API Anahtarı Yapılandırması <font color="#e06c75">🔐</font></h3>
<p>API anahtarımızı doğrudan kodun içine yazmak yerine, Colab'in güvenli <code>Secrets</code> (Gizli Anahtarlar) özelliğini kullanarak <code>GOOGLE_API_KEY</code> adıyla sakladığımız anahtarı çekiyoruz. Bu yöntem, anahtarımızın gizli ve güvende kalmasını sağlar. <code>genai.configure()</code> fonksiyonu ile bu anahtarı kullanarak Google servisleriyle güvenli bir bağlantı kuruyoruz.</p>
<br>
<h3 style="color: #e5c07b;">Dil Modeli (LLM) Testi <font color="#98c379">🚀</font></h3>
<p>Sadece anahtarı yüklemek yeterli değildir. Her şeyin yolunda olduğundan emin olmak için, projemizde kullanacağımız <strong><code>gemini-2.5-flash</code></strong> modelini çağırarak küçük bir "Merhaba!" testi yapıyoruz. Bu test, hem API anahtarımızın geçerli olduğunu hem de seçtiğimiz modele erişimimiz olduğunu anında doğrular.</p>
<blockquote style="border-left: 4px solid #c678dd; padding-left: 15px; margin: 10px 0; color: #abb2bf;">
💡 Bu hücrenin çıktısında ✅ <strong>'Her şey yolunda!'</strong> mesajını görmek, yapay zeka motorumuzun çalışmaya hazır olduğu ve projenin bir sonraki adımlarına geçebileceğimiz anlamına gelir.
</blockquote>
</div>

In [10]:
import os
from google.colab import userdata
import google.generativeai as genai

print("API anahtarı ayarlanıyor...")

try:
    api_key = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = api_key
    genai.configure(api_key=api_key)
    print("✅ API anahtarı başarıyla yüklendi.")

    print("Özel erişimli Gemini modeli ('gemini-2.5-flash') test ediliyor...")
    model_test = genai.GenerativeModel('gemini-2.5-flash')
    model_test.generate_content("Merhaba!")
    print("✅ Gemini modeli başarıyla test edildi. Her şey yolunda!")

except Exception as e:
    print(f"""
    ❌ HATA: Bir sorun oluştu.
    Eğer '404 Not Found' hatası alıyorsan, bu Google hesabının bu modele erişimi olmayabilir.
    Lütfen projeye ilk başladığın orijinal Google hesabını kullandığından emin ol.

    Teknik Hata Detayı: {e}
    """)

API anahtarı ayarlanıyor...
✅ API anahtarı başarıyla yüklendi.
Özel erişimli Gemini modeli ('gemini-2.5-flash') test ediliyor...
✅ Gemini modeli başarıyla test edildi. Her şey yolunda!


<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #e5c07b;">
<h1 style="color: #e5c07b; font-size: 32px; text-align: center; border-bottom: 2px solid #e5c07b; padding-bottom: 10px; margin-top: 0;">
💾 Bölüm 2: Bilgi Bankasının İnşası (Tek Seferlik İşlem)
</h1>
<p style="font-size: 16px;">Bu bölüm, projemizin en uzun süren adımlarını içerir. "Nörobilim Asistanı"nın "hafızası" burada sıfırdan inşa edilmiştir. Bu işlem yaklaşık 1.5 saat sürdüğü için, <strong>yalnızca bir kez çalıştırılmış</strong> ve sonuç Google Drive'a kaydedilmiştir.</p>
<blockquote style="border-left: 4px solid #e06c75; padding-left: 15px; background-color: rgba(224, 108, 117, 0.1); color: #abb2bf;">
    ⚠️ <strong>DİKKAT:</strong> Aşağıdaki kod blokları bir <strong>"kanıt"</strong> niteliğindedir ve projeyi çalıştırırken <strong>ATLANMALIDIR.</strong>
</blockquote>
</div>

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

print("--- ADIM 4: Metin Parçaları Vektörlere Dönüştürülüyor ---")

print("Embedding modeli (all-mpnet-base-v2) yükleniyor... Lütfen bekleyin.")

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

try:
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    print("✅ Embedding modeli başarıyla yüklendi.")

    sample_chunk_content = chunks[0].page_content
    vector = embeddings.embed_query(sample_chunk_content)

    print("\n--- Embedding Testi Başarılı ---")
    print(f"Örnek metin parçası (ilk 80 karakter): '{sample_chunk_content[:80]}...'")
    print(f"Oluşturulan vektörün boyutu (kaç sayıdan oluştuğu): {len(vector)}")
    print(f"Vektörün ilk 5 değeri: {vector[:5]}")

except Exception as e:
    print(f"❌ Hata: Embedding modeli yüklenemedi. Colab'de 'Runtime > Change runtime type' menüsünden GPU'nun (T4) seçili olduğundan emin olun. Hata detayı: {e}")

---
<h3 style="color: #e06c75;">2.2. Geliştirme Sürecinde Hızlı Test ⚡</h3>
<p>Nihai veritabanını oluşturma işlemi 1.5 saat gibi oldukça uzun bir süre aldığından, geliştirme döngüsünü hızlandırmak ve RAG akışının tüm adımlarını (Embedding -> Veritabanı Oluşturma -> Arama) hızlıca test edebilmek için stratejik bir ara adım uyguladık. Bu adımda, yaklaşık 300,000 parçalık tam veri setinden, sadece ilk <strong>10,000 parçayı</strong> alarak daha küçük ve yönetilebilir bir alt küme (subset) oluşturduk.</p>
<p>Aşağıdaki kod blokları bu süreci göstermektedir:</p>
<ol>
<li><strong>Veri Küçültme:</strong> Tam <code>chunks</code> listesinden <code>[:10000]</code> dilimleme (slicing) yöntemiyle ilk 10,000 elemanı seçerek <code>test_chunks</code> adında yeni bir liste oluşturulur.</li>
<li><strong>Test Veritabanı İnşası:</strong> <code>Chroma.from_documents</code> fonksiyonu bu sefer, bu küçük <code>test_chunks</code> listesiyle çağrılır. Bu işlem, tam veri setiyle saatler sürerken, bu alt küme ile sadece birkaç dakika içinde tamamlanır. Bu bize, <code>embedding</code> modelinin, <code>ChromaDB</code>'nin ve genel RAG mantığının doğru yapılandırıldığını hızlıca doğrulama imkanı tanımıştır.</li>
</ol>
<p>Bu "hızlı prototipleme" yaklaşımı, büyük ölçekli işlemi başlatmadan önce olası hataları erken bir aşamada tespit etmemizi sağlayarak projenin geliştirme sürecini önemli ölçüde verimli hale getirmiştir.</p>

In [ ]:
print("--- Hızlı Test İçin Veri Seti Küçültülüyor ---")

print(f"Orijinal parça (chunk) sayısı: {len(chunks)}")

test_chunks = chunks[:10000]

print(f"Test için kullanılacak parça sayısı: {len(test_chunks)}")

In [ ]:
print(" Vektör Veritabanı (Hızlı Test) Oluşturuluyor ---")

from langchain_chroma import Chroma
from tqdm import tqdm # ilerleme çubuğu için

persist_directory = 'neuro_coach_db_test' # Test için farklı bir klasör adı

try:
    # Veritabanını oluştururken artık 'chunks' yerine 'test_chunks' kullanıyoruz!
    vector_db_test = Chroma.from_documents(
        documents=[test_chunks[0]],
        embedding=embeddings,
        persist_directory=persist_directory
    )

    remaining_chunks_test = test_chunks[1:]
    batch_size = 100
    for i in tqdm(range(0, len(remaining_chunks_test), batch_size), desc="Test chunk'ları ekleniyor"):
        batch = remaining_chunks_test[i:i+batch_size]
        vector_db_test.add_documents(documents=batch)

    print(f"\n✅ Test veritabanı başarıyla oluşturuldu.")
    print(f"Test veritabanındaki vektör sayısı: {vector_db_test._collection.count()}")

except Exception as e:
    print(f"\n❌ Hata: {e}")

<h3 style="color: #98c379;">2.3. Nihai Veritabanının Oluşturulması 🏗️</h3>
<p>Hızlı testlerle tüm sistemin doğru çalıştığını doğruladıktan sonra, projenin en kritik ve en uzun süren adımı olan nihai bilgi bankasının inşasına geçtik. Bu adımda, <strong>tam veri setini (~300,000 parça)</strong> kullanarak, chatbot'umuzun "beynini" oluşturacak olan kalıcı vektör veritabanını inşa ediyoruz.</p>
<p>Bu süreçte, bellek (RAM) taşması gibi sorunları önlemek ve süreci izlenebilir kılmak için verimli bir yöntem izlenmiştir:</p>
<ol>
<li><strong>Başlatma (Initialization):</strong> Veritabanı, <code>Chroma.from_documents</code> komutuyla sadece <strong>tek bir "chunk"</strong> kullanılarak başlatılır. Bu, veritabanı dosyalarının diskte ilk kez oluşturulmasını sağlar.</li>
<li><strong>Toplu Ekleme (Batch Processing):</strong> Geri kalan yüz binlerce "chunk", <code>for</code> döngüsü içinde <strong>100'erli gruplar (batches)</strong> halinde veritabanına eklenir. <code>vector_db_final.add_documents()</code> metodu kullanılarak yapılan bu toplu ekleme işlemi, her bir "chunk"ı tek tek işlemekten çok daha verimlidir.</li>
<li><strong>İzleme (Monitoring):</strong> Tüm bu döngü, <code>tqdm</code> kütüphanesi ile sarmalanmıştır. Bu, işlem sırasında bize canlı bir <strong>ilerleme çubuğu</strong> sunarak, sürecin ne kadarının tamamlandığını, ne kadar sürdüğünü ve tahmini olarak ne zaman biteceğini görmemizi sağlar.</li>
</ol>
<p>Bu "toplu işleme" ve "izleme" yaklaşımı, büyük veri setleriyle çalışırken hem sistem kaynaklarını verimli kullanmanın hem de uzun süren işlemleri kontrol altında tutmanın en iyi pratiklerinden biridir.</p>

In [ ]:
from langchain_chroma import Chroma
from tqdm import tqdm
import os

print("--- NİHAİ VERİTABANI OLUŞTURMA ---")

# Oluşturulacak klasörün adı
persist_directory_final = 'neuro_coach_db_final'

# Eğer klasör daha önce oluşturulduysa, işlemi atla
if os.path.exists(persist_directory_final):
    print(f"'{persist_directory_final}' klasörü zaten mevcut. Oluşturma adımı atlanıyor.")
    # İstersen mevcut veritabanını yükleyebilirsin
    # vector_db_final = Chroma(persist_directory=persist_directory_final, embedding_function=embeddings)
else:
    # İlk chunk ile veritabanını başlat
    vector_db_final = Chroma.from_documents(
        documents=[chunks[0]], # Orijinal 'chunks' listesini kullanıyoruz
        embedding=embeddings,
        persist_directory=persist_directory_final
    )

    # Geri kalan tüm chunk'ları ilerleme çubuğu ile ekle
    remaining_chunks_final = chunks[1:]
    batch_size = 100 # 100'erli gruplar halinde
    for i in tqdm(range(0, len(remaining_chunks_final), batch_size), desc="Nihai chunk'lar ekleniyor"):
        batch = remaining_chunks_final[i:i+batch_size]
        vector_db_final.add_documents(documents=batch)

    print(f"\n✅ Nihai veritabanı başarıyla oluşturuldu.")
    print(f"Veritabanındaki toplam vektör sayısı: {vector_db_final._collection.count()}")

print("\nVeritabanı oluşturma işlemi tamamlandı.")

<h3 style="color: #56b6c2;">2.4. Bilgi Bankasını Kalıcı Hale Getirme (Yedekleme) 💾</h3>
<p>Yaklaşık 1.5 saat süren yoğun bir işlemle oluşturduğumuz vektör veritabanı, Colab'in geçici depolama alanında bulunmaktadır. Bu, Colab oturumu kapandığında bu değerli verinin kaybolacağı anlamına gelir. Bu uzun işlemi her seferinde tekrar yapmamak için, oluşturulan veritabanını kalıcı depolama alanımız olan <strong>Google Drive'a yedekliyoruz.</strong></p>
<p>Bu süreç aşağıdaki adımları içerir:</p>
<ol>
<li><strong>Sıkıştırma:</strong> <code>ChromaDB</code> tarafından oluşturulan ve çok sayıda küçük dosyadan oluşan <code>neuro_coach_db_final</code> klasörü, taşıma işlemini hızlandırmak ve tek bir dosya haline getirmek için <code>!zip</code> komutuyla sıkıştırılarak bir <code>.zip</code> arşivi oluşturulur.</li>
<li><strong>Google Drive Bağlantısı:</strong> <code>google.colab.drive</code> kütüphanesi kullanılarak Colab ortamı, kişisel Google Drive hesabımıza bağlanır.</li>
<li><strong>Kopyalama:</strong> Oluşturulan <code>.zip</code> arşivi, <code>!cp</code> komutuyla Colab'in geçici diskinden, Google Drive'ımızdaki <code>NeuroCoach_Project</code> adlı özel bir klasörün içine kopyalanır.</li>
</ol>
<p>Bu yedekleme işlemi sayesinde, projenin sonraki tüm çalıştırmalarında bu uzun "inşa" sürecini atlayıp, doğrudan Google Drive'daki bu hazır veritabanını yükleyerek projemize dakikalar içinde başlayabiliriz.</p>

In [ ]:
from google.colab import drive
import os

print("--- Oluşturulan Veritabanını Google Drive'da Özel Bir Klasöre Kaydetme ---")

# --- AYARLAR ---
drive_folder_name = "NeuroCoach_Project"
persist_directory_final = 'neuro_coach_db_final'
zip_filename = 'neuro_coach_db_final.zip'
# --- BİTTİ ---

drive_folder_path = f'/content/drive/MyDrive/{drive_folder_name}'
drive_zip_path = f'{drive_folder_path}/{zip_filename}'
colab_zip_path = f'/content/{zip_filename}'

try:
    # 1. Google Drive'ı bağla (İzin isteyecek)
    drive.mount('/content/drive', force_remount=True)

    # 2. Drive'da hedef klasörü oluştur
    if not os.path.exists(drive_folder_path):
        print(f"'{drive_folder_name}' klasörü Drive'da oluşturuluyor...")
        os.makedirs(drive_folder_path)
    else:
        print(f"'{drive_folder_name}' klasörü Drive'da zaten mevcut.")

    # 3. Klasörü sıkıştır (bu işlem birkaç dakika sürebilir)
    print(f"'{persist_directory_final}' klasörü sıkıştırılıyor...")
    !zip -r {colab_zip_path} {persist_directory_final}

    # 4. Sıkıştırılmış dosyayı Drive'a kopyala (bu işlem de birkaç dakika sürebilir)
    print(f"'{zip_filename}' dosyası hedef Drive klasörüne kopyalanıyor...")
    !cp {colab_zip_path} {drive_folder_path}/

    print(f"\n✅ BAŞARILI! Veritabanınız şu konuma kaydedildi: '{drive_zip_path}'")
    print("Artık bu uzun işlemi bir daha yapmanıza gerek yok!")

except Exception as e:
    print(f"❌ Kaydetme sırasında bir hata oluştu: {e}")

<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #56b6c2;">
<h1 style="color: #56b6c2; font-size: 32px; text-align: center; border-bottom: 2px solid #56b6c2; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
🚀 Bölüm 3: Hızlı Başlangıç - Hazır Bilgi Bankasını Yükleme
</h1>
<p style="font-size: 16px;">Bölüm 2'de bir kez çalıştırıp Google Drive'a kaydettiğimiz o devasa emek ve zamanın meyvelerini bu bölümde topluyoruz. Bu bölüm, projeyi her başlattığımızda çalıştıracağımız "hızlı başlangıç" rutinidir. 1.5 saatlik veritabanı oluşturma sürecini atlayarak, projemizi birkaç dakika içinde kullanıma hazır hale getirir.</p>
<h3 style="color: #e5c07b;">Süreç Adımları <font color="#98c379">⚙️</font></h3>
<p>Aşağıdaki kod bloğu, bu hızlı başlangıç sürecini otomatize eder:</p>
<ol>
<li><strong>Akıllı Temizlik:</strong> Kod, başlamadan önce önceki çalıştırmalardan kalmış olabilecek eski veritabanı klasörlerini ve <code>.zip</code> dosyalarını otomatik olarak siler. Bu, olası dosya çakışmalarını önler.</li>
<li><strong>Google Drive Bağlantısı:</strong> Projemizi, yedeklenmiş veritabanının bulunduğu Google Drive hesabımıza bağlar.</li>
<li><strong>Veritabanını Getirme:</strong> Drive'daki sıkıştırılmış veritabanı (<code>.zip</code>) dosyasını Colab'in hızlı yerel diskine kopyalar ve arşivi açar.</li>
<li><strong>Embedding Modelini Yükleme:</strong> Veritabanındaki vektörleri "anlayabilmek" için, o veritabanını oluştururken kullandığımız <strong>aynı</strong> <code>embedding</code> modelini tekrar hafızaya yükler. Bu, tutarlılık için kritik bir adımdır.</li>
<li><strong>Veritabanını Aktif Etme:</strong> Son olarak, <code>Chroma</code>'ya diskteki klasörün yolunu göstererek, hazır bilgi bankasını <code>beyin</code> adında bir değişkene atar ve kullanıma hazır hale getirir.</li>
</ol>
<blockquote style="border-left: 4px solid #56b6c2; padding-left: 15px; margin: 10px 0; color: #abb2bf;">
</blockquote>
</div>

In [11]:
import os
from google.colab import drive
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Kod, başlamadan önce eskiden kalma dosyaları kontrol edip silecek.
if os.path.exists('/content/neuro_coach_db_final'):
    print("Eskiden kalma veritabanı klasörü bulundu ve temizleniyor...")
    !rm -rf /content/neuro_coach_db_final

if os.path.exists('/content/neuro_coach_db_final.zip'):
    print("Eskiden kalma .zip dosyası bulundu ve temizleniyor...")
    !rm -f /content/neuro_coach_db_final.zip
# --- TEMİZLİK BİTTİ ---

# Beynin "düşünme" şeklini (embedding modelini) tekrar yükleyelim.
print("Beynin düşünme şekli yükleniyor...")
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"

)

# Şimdi Google Drive'a bağlanalım.
drive.mount('/content/drive')

# Beyin dosyasını (.zip) Drive'dan Colab'e kopyalayalım
!cp /content/drive/MyDrive/NeuroCoach_Project/neuro_coach_db_final.zip /content/

# Beyin dosyasını açalım (Artık soru sormayacak çünkü eski klasörü sildik)
!unzip -q /content/neuro_coach_db_final.zip -d /content/

# Son olarak, açtığımız beyin klasörünü kullanıma hazır hale getirelim
print("Beyin kullanıma hazırlanıyor...")
beyin = Chroma(
    persist_directory='neuro_coach_db_final',
    embedding_function=embeddings_model
)

print(f"✅ Beyin başarıyla getirildi ve kullanıma hazır! İçinde {beyin._collection.count()} adet bilgi parçacığı var.")

Eskiden kalma veritabanı klasörü bulundu ve temizleniyor...
Eskiden kalma .zip dosyası bulundu ve temizleniyor...
Beynin düşünme şekli yükleniyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Beyin kullanıma hazırlanıyor...
✅ Beyin başarıyla getirildi ve kullanıma hazır! İçinde 299942 adet bilgi parçacığı var.


<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #e06c75;">
<h1 style="color: #e06c75; font-size: 32px; text-align: center; border-bottom: 2px solid #e06c75; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
🌐 Bölüm 3.5: Çok Dilli Yeteneklerin Kazandırılması
</h1>
<p style="font-size: 16px;">Bilgi bankamız İngilizce metinlerden oluştuğu için, RAG mimarisinin arama (Retrieval) adımının verimli çalışabilmesi adına kullanıcı sorgularının da İngilizce olması gerekmektedir. Bu bölümde, chatbot'umuzu evrensel hale getirmek ve farklı dillerdeki soruları anlayabilmesini sağlamak için iki temel yardımcı fonksiyon tanımlıyoruz. Bu fonksiyonlar, Gemini modelinin dil yeteneklerini akıllıca kullanarak bir "evrensel tercüman" görevi görür.</p>
<h3 style="color: #e5c07b;">Fonksiyonlar ve Görevleri <font color="#c678dd">🛠️</font></h3>
<ol>
<li>
<strong><code>detect_language(text)</code> - Dil Tespiti:</strong>
<p>Bu fonksiyon, kullanıcıdan gelen metni alır ve Gemini modeline basit bir prompt ("Bu metin hangi dilde?") göndererek metnin dil kodunu (örn: 'tr', 'en') tespit eder. Bu, gereksiz çeviri işlemlerini önlemek için kritik bir ilk adımdır.</p>
</li>
<li>
<strong><code>translate_to_english(text, source_language)</code> - İngilizce'ye Çeviri:</strong>
<p>Eğer tespit edilen dil İngilizce değilse, bu fonksiyon devreye girer. Metni, Gemini modeline "Bu metni İngilizce'ye çevir" prompt'u ile göndererek, RAG sistemimizin arama yapabileceği standart bir formata dönüştürür. Metin zaten İngilizce ise, bu adımı atlayarak verimlilik sağlar.</p>
</li>
</ol>
<blockquote style="border-left: 4px solid #e06c75; padding-left: 15px; margin: 10px 0; color: #abb2bf;">
💡 Bu yaklaşım, ayrı bir çeviri servisi API'sine ihtiyaç duymadan, zaten kullanmakta olduğumuz güçlü Gemini modelini hem dil tespiti hem de çeviri için kullanarak projemizi daha basit ve daha entegre bir yapıda tutmamızı sağlar.
</blockquote>
</div>

In [12]:

import google.generativeai as genai

print("Çeviri ve dil tespiti için yardımcı fonksiyonlar hazırlanıyor...")

# Bu model, sadece bu hücredeki küçük işler için kullanılacak.
helper_model = genai.GenerativeModel('gemini-2.5-flash')

def detect_language(text: str) -> str:
    """Verilen metnin dil kodunu ('en', 'tr' vb.) döndürür."""
    try:
        prompt = f"What language is this text? Answer with only the two-letter ISO 639-1 code. Text: '{text}'"
        response = helper_model.generate_content(prompt)
        language_code = response.text.strip().lower()
        print(f"Tespit edilen dil: {language_code}")
        return language_code
    except Exception as e:
        print(f"Dil tespiti sırasında hata: {e}")
        return "en" # Hata olursa, varsayılan olarak İngilizce kabul et

def translate_to_english(text: str, source_language: str) -> str:
    """Verilen metni, kaynak dilini bilerek İngilizce'ye çevirir."""
    if "en" in source_language:
        print("Metin zaten İngilizce, çeviri atlanıyor.")
        return text

    try:
        print(f"'{source_language}' dilinden İngilizce'ye çevriliyor...")
        prompt = f"Translate the following text to English. Respond with only the translated text: '{text}'"
        response = helper_model.generate_content(prompt)
        translated_text = response.text.strip()
        print(f"Çeviri: {translated_text}")
        return translated_text
    except Exception as e:
        print(f"Çeviri sırasında hata: {e}")
        return text

print("✅ Yardımcı fonksiyonlar hazır.")

Çeviri ve dil tespiti için yardımcı fonksiyonlar hazırlanıyor...
✅ Yardımcı fonksiyonlar hazır.


<div style="background-color: #282c34; padding: 20px; border-radius: 10px; border: 1px solid #98c379;">
<h1 style="color: #98c379; font-size: 32px; text-align: center; border-bottom: 2px solid #98c379; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
⚙️ Bölüm 4: Ana Chatbot Motorunun İnşası
</h1>
<p style="font-size: 16px;">Bu bölümde, projemizin kalbini ve beynini oluşturan ana RAG (Retrieval Augmented Generation) motorunu inşa ediyoruz. Bu motor, hem önceki konuşmaları hatırlayabilen (hafızalı) hem de farklı dillerde talimat alabilen (çok dilli) gelişmiş bir yapıya sahiptir. Bu adımların sonunda, Gradio arayüzüne bağlanmaya hazır, tam işlevsel bir chatbot mantığımız olacak.</p>
<h3 style="color: #e5c07b;">4.1. Temel Bileşenlerin Hazırlanması <font color="#61afef">🧩</font></h3>
<p>Zincirimizi oluşturmadan önce, onu oluşturan ana parçaları hazırlıyoruz:</p>
<ul>
<li><strong>Retriever (Arama Yapıcı):</strong> Bir önceki bölümde yüklediğimiz <code>beyin</code> (ChromaDB veritabanı) değişkenini, LangChain'in anlayacağı bir "arama motoruna" dönüştürüyoruz. Her sorguda en alakalı 5 belgeyi getirecek şekilde ayarlanmıştır.</li>
<li><strong>LLM (Dil Modeli):</strong> Cevapları üretecek olan <code>gemini-2.5-flash</code> modelini LangChain ile uyumlu hale getiriyoruz.</li>
<li><strong>Memory (Hafıza):</strong> <code>ConversationBufferMemory</code> ile chatbot'umuzun sohbet geçmişini saklayacağı geçici bir hafıza alanı oluşturuyoruz.</li>
</ul>
<br>
<h3 style="color: #e5c07b;">4.2. Çok Dilli Prompt Şablonları <font color="#c678dd">🌍</font></h3>
<p>Chatbot'umuzun, kullanıcının sorduğu dile göre cevap verebilmesi için iki ayrı "görev tanımı" (prompt şablonu) hazırlıyoruz. Bir tanesi Türkçe cevap üretmesi için talimatlar içerirken, diğeri İngilizce cevap üretmesi için talimatlar içerir. Bu şablonlar, bir sonraki bölümde (Gradio) kullanıcının diline göre dinamik olarak seçilip zincire enjekte edilecektir.</p>
<br>
<h3 style="color: #e5c07b;">4.3. Hafızalı Ana Zincirin Oluşturulması <font color="#98c379">🔗</font></h3>
<p>Tüm parçaları, LangChain'in güçlü ve hazır zincirlerinden biri olan <code>ConversationalRetrievalChain</code> ile birleştiriyoruz. Bu zincir, bizim için birçok karmaşık işi otomatik olarak yapar:</p>
<ol>
<li>Yeni soruyu ve <strong>hafızadaki eski konuşmaları</strong> birleştirerek daha akıllı bir arama sorgusu oluşturur.</li>
<li>Bu sorguyla <strong>Retriever</strong>'ı kullanarak veritabanında arama yapar.</li>
<li>Bulunan belgeleri, sohbet geçmişini ve yeni soruyu <strong>LLM</strong>'e göndererek nihai cevabı üretir.</li>
<li>Yeni soru ve cevabı otomatik olarak <strong>hafızaya</strong> ekler.</li>
<li><code>return_source_documents=True</code> parametresi sayesinde, cevabı üretirken kullandığı kaynak belgeleri de bize sunar.</li>
</ol>
<blockquote style="border-left: 4px solid #98c379; padding-left: 15px; margin: 10px 0; color: #abb2bf;">
💡 Bu "hazır zincir" yaklaşımı, karmaşık LCEL yapıları kurmadan, hafıza yönetimini ve RAG akışını son derece basit ve okunabilir bir kodla uygulamamızı sağlar.
</blockquote>
</div>

In [13]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

print("Hafızalı temel motor ve çok dilli prompt'lar hazırlanıyor...")

# 1. Temel Bileşenler
arama_yapici = beyin.as_retriever(search_kwargs={"k": 5})
llm_for_chain = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

# 2. Hafıza Nesnesi
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
    output_key='answer'
)

# 3. FARKLI DİLLER İÇİN PROMPT ŞABLONLARI
# Bu prompt'lar, zincirin içine sonradan enjekte edilecek.
prompt_tr_template = """Sen 'Nörobilim asistanısın. Konuşma geçmişini ({chat_history}) dikkate alarak, sana verilen yeni bilgileri ({context}) kullanarak kullanıcının yeni sorusuna ({question}) TÜRKÇE cevap ver. Cevabın yardımsever ve anlaşılır olsun."""
prompt_en_template = """You are 'Neuroscience assistant'. Considering the chat history ({chat_history}), answer the user's new question ({question}) in ENGLISH, using the new context ({context}). Be helpful and clear."""

prompt_tr = PromptTemplate.from_template(prompt_tr_template)
prompt_en = PromptTemplate.from_template(prompt_en_template)


# 4. TEMEL Hafızalı Zincir
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_for_chain,
    retriever=arama_yapici,
    memory=memory,
    output_key='answer',
    return_source_documents=True
    # -------------------------
)

print("✅ Kaynak gösterebilen, hafızalı temel motor başarıyla oluşturuldu.")

print("\n--- Test Aşaması ---")
soru1 = "Erteleme (procrastination) nedir?"
result = conversational_chain.invoke({"question": soru1})

print("CEVAP:", result['answer'])
print("KAYNAK SAYISI:", len(result['source_documents']))

Hafızalı temel motor ve çok dilli prompt'lar hazırlanıyor...
✅ Kaynak gösterebilen, hafızalı temel motor başarıyla oluşturuldu.

--- Test Aşaması ---
CEVAP: Erteleme (procrastination), önemli görevleri isteyerek erteleyip, daha az önemli veya keyifli aktivitelere yönelme eylemidir.

Metne göre erteleme:
*   Dikkatimizi gerektiren görevleri geciktirme veya erteleme eylemidir.
*   Daha fazla öneme veya aciliyete sahip bir göreve yönelmek yerine, daha az önemli veya keyifli bir şeyi yapmayı gönüllü olarak seçme davranışıdır.
*   Motivasyon eksikliği, kötü zaman yönetimi becerileri, başarısızlık veya başarı korkusu gibi faktörlerden kaynaklanabilir.
*   Popüler inanışın aksine, zaman yönetimi veya tembellikle ilgili olmayıp, gelecekteki olaylar hakkında kötü duygusal düzenlemeyle ilgilidir.
KAYNAK SAYISI: 5


<div style="background-color: #282828; padding: 20px; border-radius: 10px; border: 1px solid #d3869b;">
<h1 style="color: #d3869b; font-size: 32px; text-align: center; border-bottom: 2px solid #d3869b; padding-bottom: 10px; margin-top: 0; text-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
🎨 Bölüm 5: İnteraktif Web Arayüzü (Gradio)
</h1>
<p style="font-size: 16px;">Projemizin son adımında, önceki bölümlerde inşa ettiğimiz güçlü ve akıllı chatbot motorunu, kullanıcıların kolayca etkileşime girebileceği modern ve şık bir web arayüzü ile buluşturuyoruz. Bu, projemizin teknik yeteneklerini son kullanıcıya sunan en önemli vitrinidir.</p>
<h3 style="color: #83a598;">5.1. Arayüz Tasarımı ve Felsefesi ✨</h3>
<p>Arayüzün tasarımında, projenin "nörobilim" ve "teknoloji" temalarını yansıtan, fütüristik ve kullanıcı dostu bir yaklaşım benimsenmiştir:</p>
<ul>
<li><strong>Özel Tasarım (CSS):</strong> Arayüz, standart Gradio görünümünün dışına çıkarak, özel CSS kodları ile tamamen kişiselleştirilmiştir. Kozmik temalı bir arkaplan resmi, "buzlu cam" efekti veren yarı şeffaf ana panel ve markalaşma için özel logo ve font kullanımı gibi modern web tasarım teknikleri uygulanmıştır.</li>
<li><strong>Bileşen Yerleşimi (<code>gr.Blocks</code>):</strong> Standart <code>ChatInterface</code> yerine, arayüz bileşenleri (logo, başlık, sohbet penceresi, metin kutusu) üzerinde tam kontrol sağlayan <code>gr.Blocks</code> yapısı kullanılmıştır. Bu sayede, logo ve başlık yan yana hizalanarak profesyonel bir görünüm elde edilmiştir.</li>
</ul>
<br>
<h3 style="color: #83a598;">5.2. Arayüz Mantığı ve Çalışma Akışı 🧠</h3>
<p>Görsel tasarımın arkasında, önceki bölümlerde oluşturduğumuz tüm RAG mantığını yöneten akıllı bir fonksiyon yatmaktadır:</p>
<ol>
<li><strong><code>get_neuro_coach_response()</code> Fonksiyonu:</strong> Bu "ana beyin" fonksiyonu, Gradio arayüzünden gelen her bir kullanıcı mesajını alır.</li>
<li><strong>Çok Dilli İşlem:</strong> Fonksiyon, Bölüm 3.5'teki yardımcı fonksiyonları çağırarak mesajın dilini tespit eder ve gerekirse İngilizce'ye çevirir.</li>
<li><strong>Dinamik Prompting:</strong> Bölüm 4'te hazırlanan, dile özel prompt şablonlarından uygun olanı seçer ve hafızalı ana zincirin (<code>conversational_chain</code>) görev tanımını o anki konuşmanın diline göre günceller.</li>
<li><strong>Cevap Üretimi:</strong> Güncellenmiş zinciri çalıştırarak nihai, hem hafızalı hem de doğru dilde cevabı üretir ve arayüze geri döndürür.</li>
</ol>
<blockquote style="border-left: 4px solid #d3869b; padding-left: 15px; margin: 10px 0; color: #d4be98;">
🚀 <strong>Sonuç:</strong> Bu hücre çalıştırıldığında, herkesle paylaşılabilen bir "Public URL" oluşturulur. Bu link üzerinden erişilen arayüz, projemizin tüm yeteneklerini (hafıza, çok dillilik, RAG) son kullanıcıya sunan, tam işlevsel bir web uygulamasıdır.
</blockquote>
</div>

In [ ]:
import gradio as gr
import time
import re

print("Projenin nihai arayüzü inşa ediliyor...")

# --- GÖRSEL TASARIM (CSS ve URL'ler) ---
LOGO_URL = "https://github.com/Fatmanurkntr/Neuroscience-Chatbot/blob/main/chatbotlogo1.png?raw=true"
BACKGROUND_IMAGE_URL = "https://github.com/Fatmanurkntr/Neuroscience-Chatbot/blob/main/neuro.jpg?raw=true"

# Tüm görsel ayarları içeren CSS kodu.
custom_css = f"""
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600;700&display=swap');

body, .gradio-container {{
    font-family: 'Poppins', sans-serif;
    background-image: url('{BACKGROUND_IMAGE_URL}');
    background-size: cover;
    background-position: center;
    background-attachment: fixed;
}}

#app_container {{
    max-width: 800px;
    margin: 40px auto;
    background-color: rgba(20, 30, 48, 0.75);
    backdrop-filter: blur(15px);
    border-radius: 20px;
    padding: 30px;
    border: 1px solid rgba(255, 255, 255, 0.1);
}}

/* Başlığı saran ana konteyner için Flexbox ayarları */
#header_container {{
    display: flex !important;
    flex-direction: row !important;
    justify-content: center !important;
    align-items: center !important;
    margin-bottom: 25px !important;
    gap: 20px !important;
}}

/* Logo'nun kendisi için boyut ayarı */
#header_logo img {{
    width: 80px !important;
    height: 80px !important;
    min-width: 80px !important;
}}

/* Metinleri saran grup için ayarlar */
#header_text_group {{
    display: flex !important;
    flex-direction: column !important;
    align-items: flex-start !important;
}}

/* Ana Başlık ve Alt Başlık stilleri */
#header_title {{ color: #FFFFFF; font-size: 32px; font-weight: 700; margin: 0; padding: 0; line-height: 1.2; }}
#header_subtitle {{ color: #D1D5DB; font-size: 16px; margin-top: 5px; padding: 0; }}

/* Sohbet baloncukları ve diğer stiller */
#main_chatbot .user {{ background: linear-gradient(to right, rgba(79, 70, 229, 0.8), rgba(124, 58, 237, 0.8)) !important; }}
#main_chatbot .bot {{ background-color: rgba(55, 65, 81, 0.8) !important; }}
"""

# --- ANA MANTIK FONKSİYONU (Gradio için özel) ---
def get_neuro_coach_response(message: str, history: list):
    """
    Bu "ana beyin" fonksiyonu, Gradio'dan gelen her soru için tüm RAG sürecini yönetir.
    """
    print("\n" + "="*50)
    print(f"--- Arayüzden Gelen Soru: {message} ---")

    try:
        language = detect_language(message)
        english_question = translate_to_english(message, language)

        if language == "tr":
            selected_prompt = prompt_tr
        else:
            selected_prompt = prompt_en

        conversational_chain.combine_docs_chain.llm_chain.prompt = selected_prompt

        result = conversational_chain.invoke({"question": english_question})
        answer = result.get('answer', "Üzgünüm, bir cevap üretemedim.")

        print(f"--- Üretilen Cevap ---")
        return answer

    except Exception as e:
        print(f"!!!!!! HATA OLUŞTU: {e} !!!!!!")
        return f"Üzgünüm, bir hata oluştu: {e}"

# --- 3. ARAYÜZÜN İNŞASI VE CANLANDIRILMASI ---
with gr.Blocks(css=custom_css, theme=None, title="Nörobilim Asistanı") as demo:

    with gr.Column(elem_id="app_container"):

        # Başlık Bölümü
        with gr.Row(elem_id="header_container"):
            gr.Image(value=LOGO_URL, elem_id="header_logo", show_download_button=False, container=False, show_fullscreen_button=False, scale=0)
            with gr.Column(elem_id="header_text_group", scale=1):
                gr.Markdown("# Nörobilim Asistanı", elem_id="header_title")
                gr.Markdown("Zihninizin karmaşık haritasını keşfedin. Motivasyon, erteleme ve alışkanlıklarınızın ardındaki nörobilimsel sırları aydınlatmak için bir soru sorun.", elem_id="header_subtitle")

        # 1. Sohbet penceresi
        chatbot = gr.Chatbot(height=450, render_markdown=True, elem_id="main_chatbot", label="Sohbet")

        # 2. Mesaj yazma kutusu
        msg = gr.Textbox(placeholder="Örn: 'Erteleme alışkanlığından nasıl kurtulurum?'", show_label=False)

        # 3. Örnekler
        gr.Examples(
            ["Erteleme nedir?", "Peki bununla nasıl başa çıkabilirim?", "How to build a new habit?"],
            inputs=msg,
            label="Denemek için bir soru seçin:"
        )

        # Arayüz Mantığı
        def user(user_message, history):
            return "", history + [[user_message, None]]

        def bot(history):
            user_message = history[-1][0]
            bot_message = get_neuro_coach_response(user_message, history[:-1])
            history[-1][1] = bot_message
            return history

        msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
            bot, chatbot, chatbot
        )

# Arayüzü Başlatma
print("Arayüz başlatılıyor...")
demo.launch(share=True, debug=True)

# Kapanmayı Önleme
print("Gradio sunucusu aktif. Bu hücreyi canlı tutmak için çalışmaya devam edecek.")
while True:
    time.sleep(1)